# Flight Data Exploration

* Created live on stream.
Watch the video here: https://youtu.be/xs_L6z9QNYY

In [ ]:
import sys

# Print the Python version
print(sys.version)

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

# Setting display options
pd.set_option("display.max_columns", 500)

# Setting plotting style
plt.style.use("seaborn-colorblind")
pal = sns.color_palette()


In [ ]:
# List Parquet files in the directory
parquet_files = glob("../input/flight-delay-dataset-20182022/*.parquet")

# Print the list of Parquet files
parquet_files

In [ ]:
# List CSV files in the directory
csv_files = glob("../input/flight-delay-dataset-20182022/*.csv")

# Print the list of CSV files
csv_files

# Read in and Format Data

In [ ]:
# List Parquet files in the directory
parquet_files = glob("../input/flight-delay-dataset-20182022/*.parquet")

# Print the list of Parquet files
parquet_files

In [ ]:
# Define the subset of columns to keep
column_subset = [
    "FlightDate",  # Flight date
    "Airline",  # Airline code
    "Flight_Number_Marketing_Airline",  # Flight number
    "Origin",  # Origin airport code
    "Dest",  # Destination airport code
    "Cancelled",  # Indicator for canceled flights
    "Diverted",  # Indicator for diverted flights
    "CRSDepTime",  # Scheduled departure time
    "DepTime",  # Actual departure time
    "DepDelayMinutes",  # Departure delay in minutes
    "OriginAirportID",  # Origin airport ID
    "OriginCityName",  # Origin city name
    "OriginStateName",  # Origin state name
    "DestAirportID",  # Destination airport ID
    "DestCityName",  # Destination city name
    "DestStateName",  # Destination state name
    "TaxiOut",  # Taxi out time in minutes
    "TaxiIn",  # Taxi in time in minutes
    "CRSArrTime",  # Scheduled arrival time
    "ArrTime",  # Actual arrival time
    "ArrDelayMinutes",  # Arrival delay in minutes
]

# Initialize an empty list to store DataFrames
dfs = []

# Iterate over Parquet files
for f in parquet_files:
    # Read Parquet file and select specified columns
    dfs.append(pd.read_parquet(f, columns=column_subset))

# Concatenate DataFrames into a single DataFrame
df = pd.concat(dfs).reset_index(drop=True)

# Define categorical columns
cat_cols = ["Airline", "Origin", "Dest", "OriginStateName", "DestStateName"]

# Convert specified columns to categorical data type
for c in cat_cols:
    df[c] = df[c].astype("category")


# Understand the `DepDelayMinutes` Variable

In [ ]:
# Filter the DataFrame for flights with departure delays less than 30 minutes,
# select the 'DepDelayMinutes' column, and plot a histogram
df.query("DepDelayMinutes < 30")["DepDelayMinutes"].plot(
    kind="hist", bins=30, title="Distribution of Flight Delays < 30 Min"
)

# Display the plot
plt.show()

In [ ]:
# Filter the DataFrame for flights with departure delays between 1 and 60 minutes,
# select the 'DepDelayMinutes' column, and plot a histogram
df.query("DepDelayMinutes > 1 and DepDelayMinutes < 61")["DepDelayMinutes"].plot(
    kind="hist", bins=30, title="Distribution of Flight Delays (1-60 Min)"
)

# Display the plot
plt.show()

# Grouping of Delays
Per Wikipedia https://en.wikipedia.org/wiki/Flight_cancellation_and_delay:
```
Delays are divided into three categories, namely "on time or small delay" (up to 15 minutes delay), "Medium delay" (15 – 45 minutes delay) and "Large delay" ( 45 minutes delay). 
```

In [ ]:
# Create a new column named "DelayGroup" and initialize it to None
df["DelayGroup"] = None

# Assign delay groups based on the value of "DepDelayMinutes" and whether the flight was cancelled
df.loc[df["DepDelayMinutes"] == 0, "DelayGroup"] = "OnTime_Early"
df.loc[(df["DepDelayMinutes"] > 0) & (df["DepDelayMinutes"] <= 15), "DelayGroup"] = "Small_Delay"
df.loc[(df["DepDelayMinutes"] > 15) & (df["DepDelayMinutes"] <= 45), "DelayGroup"] = "Medium_Delay"
df.loc[df["DepDelayMinutes"] > 45, "DelayGroup"] = "Large_Delay"
df.loc[df["Cancelled"], "DelayGroup"] = "Cancelled"

In [ ]:
# Visualizing the count of flights in each delay group using a horizontal bar plot
df["DelayGroup"].value_counts(ascending=True).plot(
    kind="barh",  # Horizontal bar plot
    figsize=(10, 5),  # Figure size
    color=pal[1],  # Color palette for the bars
    title="Flight Results (2018-2022)"  # Plot title
)
plt.show()  # Display the plot

# How Many Flights Per Year?

In [ ]:
# Extracting the year from the FlightDate column and counting the number of flights per year
df["Year"] = df['FlightDate'].dt.year

# Plotting the number of scheduled flights per year using a bar plot
df["Year"].value_counts().sort_index().plot(
    kind="bar",  # Bar plot
    figsize=(10, 5),  # Figure size
    title="Scheduled Flights Per Year"  # Plot title
)
plt.show()  # Display the plot

# What is the % of Flight Results by Year

In [ ]:
# Extracting the year from the FlightDate column
df["Year"] = df["FlightDate"].dt.year

# Grouping the data by Year and DelayGroup, and calculating the percentage of each DelayGroup for each year
df_agg = df.groupby("Year")["DelayGroup"].value_counts(normalize=True).unstack() * 100

# Defining the order of columns
col_order = ["OnTime_Early", "Small_Delay", "Medium_Delay", "Large_Delay", "Cancelled"]

# Applying a background gradient to the DataFrame and displaying it
df_agg[col_order].style.background_gradient(cmap="Greens")


# Results by Month

In [ ]:
# Extracting the month from the FlightDate column
df["Month"] = df["FlightDate"].dt.month

# Grouping the data by Month and DelayGroup, and calculating the percentage of each DelayGroup for each month
df_agg = df.groupby("Month")["DelayGroup"].value_counts(normalize=True).unstack() * 100

# Defining the order of columns
col_order = ["OnTime_Early", "Small_Delay", "Medium_Delay", "Large_Delay", "Cancelled"]

# Applying a background gradient to the DataFrame and displaying it
df_agg[col_order].style.background_gradient(cmap="Blues")


## Exclude 2020

In [ ]:
# Extracting the month from the FlightDate column
df["Month"] = df["FlightDate"].dt.month

# Filtering out the data for the year 2020, then grouping by Month and DelayGroup,
# and calculating the percentage of each DelayGroup for each month
df_agg = (
    df.query("Year != 2020")
    .groupby("Month")["DelayGroup"]
    .value_counts(normalize=True)
    .unstack()
    * 100
)

# Defining the order of columns
col_order = ["OnTime_Early", "Small_Delay", "Medium_Delay", "Large_Delay", "Cancelled"]

# Applying a background gradient to the DataFrame and displaying it
df_agg[col_order].style.background_gradient(cmap="Blues")

## Why High Cancellation in March/April? What does it look like by year for these months?

In [ ]:
# Extracting the month from the FlightDate column
df["Month"] = df["FlightDate"].dt.month

# Filtering out the data for the months March and April, then grouping by Year and DelayGroup,
# and calculating the percentage of each DelayGroup for each year
df_agg = (
    df.query("3 <= Month <= 4")
    .groupby("Year")["DelayGroup"]
    .value_counts(normalize=True)
    .unstack()
    * 100
)

# Defining the order of columns
col_order = ["OnTime_Early", "Small_Delay", "Medium_Delay", "Large_Delay", "Cancelled"]

# Applying a background gradient to the DataFrame along the rows (axis=0) and displaying it
df_agg[col_order].style.background_gradient(cmap="Oranges", axis=0)

# Plot using `calmap`

In [ ]:
!pip install calmap plotly_calplot -q

In [ ]:
import calmap

# Grouping the data by FlightDate and calculating the mean of Cancelled flights for each date
events = df.groupby("FlightDate")["Cancelled"].mean()

# Creating subplots for each year from 2018 to 2022
fig, axs = plt.subplots(5, 1, figsize=(10, 10))

# Looping through each year and plotting the yearly calendar heatmap
for i, year in enumerate([2018, 2019, 2020, 2021, 2022]):
    # Plotting the calendar heatmap for the current year
    calmap.yearplot(
        events.apply(np.log),  # Applying logarithmic scale to the event data
        year=year,
        cmap="YlOrRd",  # Using Yellow-Orange-Red colormap
        monthly_border=True,  # Showing monthly borders
        ax=axs[i],  # Assigning the current axis
    )
    axs[i].set_title(year)  # Setting the title for the subplot

# Setting the background color of the figure to white
fig.patch.set_facecolor("white")

# Adding a title for the entire figure
fig.suptitle("US Flight Cancellations", y=0.92, fontsize=20)


# Interactive Calendar Heatmap

In [ ]:
# Importing the calplot function from plotly_calplot library
from plotly_calplot import calplot

# Creating the plot using the calplot function
# Here, we are applying the natural logarithm (np.log) to the events dataframe 
# and resetting the index before passing it to calplot
# x="FlightDate" specifies the column containing the date values
# y="Cancelled" specifies the column containing the cancelled flight data
fig = calplot(events.apply(np.log).reset_index(), x="FlightDate", y="Cancelled")

# Displaying the plot
fig.show()

# Compare Airlines
- Who has the most delays?
- Who has the most cancellations?
- Who is the most reliable? (on time)

In [ ]:
import matplotlib.pyplot as plt

# Creating a figure and axis object with specified figsize
fig, ax = plt.subplots(figsize=(10, 10))

# Calculating the number of flights for each airline and plotting as a horizontal bar chart
# Here, we are dividing the value counts of each airline by 100,000 for better visualization
# kind="barh" specifies a horizontal bar chart
# ax=ax specifies the axis object to plot on
# color=pal[2] specifies the color palette to use
# width=1 specifies the width of the bars
# edgecolor="black" specifies the color of the edges of the bars
(df["Airline"].value_counts(ascending=True) / 100_000).plot(
    kind="barh", ax=ax, color=pal[2], width=1, edgecolor="black"
)

# Setting the title of the plot
ax.set_title("Number of Flights in Dataset")

# Setting the label for the x-axis
ax.set_xlabel("Flights (100k)")

# Displaying the plot
plt.show()

## Subset to the Top Airlines
- Minimum 1M Flights

In [ ]:
# Calculating the top airlines with more than 1,000,000 flights
top_airlines = (
    df["Airline"]
    .value_counts()
    .reset_index()
    .query("Airline > 1_000_000")["index"]
    .values.tolist()
)

# Filtering the dataframe to include only flights from the top airlines
df_top = df.loc[df["Airline"].isin(top_airlines)].reset_index(drop=True).copy()

In [ ]:
# Resetting airline categories
df_top["Airline"] = df_top["Airline"].astype("str").astype("category")

In [ ]:
# Define the order of delay groups for visualization
col_order = ["OnTime_Early", "Small_Delay", "Medium_Delay", "Large_Delay", "Cancelled"]

# Group the data by airline and delay group, calculate the percentage of flights in each delay group for each airline
df_agg = (
    df_top.groupby(["Airline"])["DelayGroup"]
    .value_counts(normalize=True)
    .unstack()[col_order]
)

# Create a horizontal bar plot to visualize the flight result breakdown for top airlines
fig, ax = plt.subplots(figsize=(10, 5))
df_agg.sort_values("OnTime_Early").plot(
    kind="barh", stacked=True, ax=ax, width=0.8, edgecolor="black"
)
ax.legend(bbox_to_anchor=(1, 1))
ax.set_title("Top Airlines Flight Result Breakdown", fontsize=20)
ax.set_xlabel("Percent of Total Flights")
plt.show()

## Split out Each Delay Type and plot

In [ ]:
# Define a mapping dictionary for delay groups
delay_mapping = {
    "OnTime_Early": "Good",
    "Small_Delay": "Good",
    "Medium_Delay": "Bad",
    "Large_Delay": "Bad",
    "Cancelled": "Bad",
}

In [ ]:
# Map delay groups to a new column indicating whether the delay is considered acceptable or not
df_top["DelayOk"] = df_top["DelayGroup"].map(delay_mapping)

In [ ]:
# Group by airline and the "DelayOk" column, calculate the normalized value counts, and unstack the result
df_agg = (
    df_top.groupby(["Airline"])["DelayOk"]
    .value_counts(normalize=True)
    .unstack()[["Good", "Bad"]]
)

# Plot the results as a stacked horizontal bar chart
fig, ax = plt.subplots(figsize=(10, 5))
df_agg.sort_values("Good").plot(
    kind="barh", stacked=True, ax=ax, width=0.8, edgecolor="black"
)
ax.legend(bbox_to_anchor=(1, 1))
ax.set_title("Top Airlines Flight Result Ok vs Not Ok", fontsize=20)

plt.show()

# Done!!!